In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt 
import seaborn as sns
import sklearn
from Bio.Seq import Seq
np.random.seed(42)
from transformers import TFBertModel, BertTokenizer,BertConfig, AutoTokenizer, TFAutoModelForSequenceClassification, AdamWeightDecay
import re
np.random.seed(42)
tf.random.set_seed(42)
import pickle
import sys
import gc


In [2]:
df = pd.read_csv("../Data/CoV-AbDab_031022.csv")
# df.reset_index(drop=True, inplace=True)
df = df[["VHorVHH"]]
df = df[df["VHorVHH"].apply(lambda x: len(x) <= 138)]
df = df[(df.VHorVHH != 'ND')]
df.rename(columns={'VHorVHH': 0}, inplace=True)
df.insert(1, "label", [1] * 11415)
df.to_csv("covid_df_test.csv",index=False, header=None)
# df = df[["CDRH3"]]

In [3]:
dummy = []
head = []
with open("../Data/cAb-rep/cAb-Rep_heavy.nt.txt") as myfile:
    # count = 0
    for i in myfile:
        # if count <= 1:
        #     print(i)
        #     if i.find(">") == -1 & i.find("-") == -1:
        #         print(Seq.translate(i.strip()))
        #     count+=1
        dummy.append(i)
    np.random.shuffle(dummy)
    
    for i in dummy:
        if i.find(">") == -1 & i.find("-") == -1 & i.find("N") == -1: # These conditions must be met for a valid sequence, the longest was 141. However, there is no 141 sequence for COVID, the greatest is 138, so we go with that
            aa_sequence = Seq.translate(i.strip())
            if (len(aa_sequence) <= 138) & (len(aa_sequence) >= 100):
                head.append(aa_sequence)
                if len(head) >= 11415:
                    break
print(head[:5], len(head))
healthy_sequences = head

/Users/joseph/miniforge3/envs/test_env/lib/python3.10/site-packages/Bio/Seq.py:3482: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn(


['EVQLVQSGPEVKKPGSSVKVSCKASGGTFSNFAFSWVRQAPGQGLEWMGSVILHLGTSTYAQKFQGRVTITADESTSAAFMDLNALTSDDTAVYYCARVVAVPGRVPYWFDPWGQGTLVTVSS', 'TLSLTCAVYGGSFSGYYWSWIRQPPGKGLEWIGEINHSGSTNYNPSLKSRVTISVDTSKNQFSLKLSSVTAADTAVYYCARVPPTSTVTTLGDDYWGQGTLVTVSS', 'QVQLVQSGPEVKKPGASVRVSCKPSGYPFSNYGISWMRQAPGQGLEWMGWVNIDKGNTKYAQKFQDRVTMTTDTSSSTVYLELRSLRSDDTALYYCARERGGYRYGDYWGQGTLVIVSS', 'TLSLTCAVYGGSFSGYYWSWIRQPPGKGLEWIGEIKHSGSTNYIPSLKSRVTISVDTSKNQFSLKLSSVTAADTAVYYCASRAGAAAASWGQGTLVTVSS', 'SETLSLTCAVHGGSFSDYYWTWIRQPPGKGLEWIGEINHRGGTNYNPSLKSRLNILVDTSKSQFSLKLSSVTAADTAVYFCARERFILIRGLTKYYYYMDVWGKGTTVTVS'] 11415


In [17]:
df1 = pd.DataFrame(healthy_sequences)
df1.insert(1, "label", [0] * 11415)
df1.to_csv("healthy_df_test.csv",index=False, header=None)
# df = df[["CDRH3"]]
df1

,0,label
0,EVQLVQSGPEVKKPGSSVKVSCKASGGTFSNFAFSWVRQAPGQGLE...,0
1,TLSLTCAVYGGSFSGYYWSWIRQPPGKGLEWIGEINHSGSTNYNPS...,0
2,QVQLVQSGPEVKKPGASVRVSCKPSGYPFSNYGISWMRQAPGQGLE...,0
3,TLSLTCAVYGGSFSGYYWSWIRQPPGKGLEWIGEIKHSGSTNYIPS...,0
4,SETLSLTCAVHGGSFSDYYWTWIRQPPGKGLEWIGEINHRGGTNYN...,0
...,...,...
11410,TLSLTCAVSGYSINNAYFWGWIRQPPGGGLEWIGSLYHSGSTYYTP...,0
11411,EVQLVESGTEVKKPGASVKVSCKTSGYTFKSYGIIWVRQAPGQGLE...,0
11412,RQLVQSGAEVKKPGASVKVSCKASGYTFTGYYMHWVRQAPGQGLEW...,0
11413,WSLRLSCAASGFTFSDYYMTWIRQAPGKGLEWVSYISGSNSFINYG...,0


In [18]:
df2 = pd.concat([df1,df], ignore_index=True)
df2

,0,label
0,EVQLVQSGPEVKKPGSSVKVSCKASGGTFSNFAFSWVRQAPGQGLE...,0
1,TLSLTCAVYGGSFSGYYWSWIRQPPGKGLEWIGEINHSGSTNYNPS...,0
2,QVQLVQSGPEVKKPGASVRVSCKPSGYPFSNYGISWMRQAPGQGLE...,0
3,TLSLTCAVYGGSFSGYYWSWIRQPPGKGLEWIGEIKHSGSTNYIPS...,0
4,SETLSLTCAVHGGSFSDYYWTWIRQPPGKGLEWIGEINHRGGTNYN...,0
...,...,...
22825,EVQVVESGGGLVKPGGSLRLSCAASGFTFSSYTMNWVRQAPGKGLE...,1
22826,QMQLVQSGPEVKRPGTSVKVSCEASGFTFSSSAILWVRQPRGQRLE...,1
22827,QVQLVESGGGLVKPGGSLRLSCAASGFTFSDYYMNWIRQAPGKGLE...,1
22828,EVQLVESGGGLVQPGGSLRLSCAASGFTFSRFAMHWVRQAPGKGLE...,1


In [19]:
df2 = df2.sample(frac=1)
df2

,0,label
5334,RSLRLSCVASGFAFDDHAMHWVRQAPGKGLEWVSLIDGDGLSTHYA...,0
17683,QVQLVESGGGVVQPGRSLRLSCAASGFTFRIYAMHWVRQAPGKGLE...,1
10115,QVQLVQSGAEVKKAGASVQVSCKASGYTFSEYGISWVRQAPGQGLE...,0
15530,QITLKESGPTLVKPTQTLTLTCTFSGFSLTTSGVGVGWIRQPPGKA...,1
12484,QVQLVQSGVEVKKPGASVKISCKASGYTFTDYYMHWVRQAPGQGLE...,1
...,...,...
1050,TLSLTCTVSGDSITDYHWSWIRQPAGRGLEWIGHFYTSGNTNYNPS...,0
2631,WSLRLSCAASGFTFSSYGMHWVRQAPGKGLEWVAVIWYDGSNKYYA...,0
4088,QSGAEVKKPGASVKVSCKASGYSFTGYYMHWVRQAPGQGLEWMGRI...,0
22190,QVTLRESGPALVKPTQTLTLTCTFSGFSLSTSGMSVNWIRQPPGEA...,1


In [20]:
df_training = df2.iloc[:19406]
df_testing = df2.iloc[19406:]

In [25]:
df_training.loc[0] = df_training[0].apply(lambda s: (" ".join(s)))

In [26]:
df_testing.loc[0] = df_testing[0].apply(lambda s: (" ".join(s)))

/var/folders/tk/3b95knbd4yjfxbnw6d207nlc0000gp/T/ipykernel_9725/3355594592.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_testing.loc[0] = df_testing[0].apply(lambda s: (" ".join(s)))


In [27]:
df_training.to_csv("../Data/ProtTrans_Finetune/healthy_vs_covid_train.csv", index=False, header=None)
df_testing.to_csv("../Data/ProtTrans_Finetune/healthy_vs_covid_test.csv", index=False, header=None)